In [208]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pprint
from dotenv import load_dotenv
from pymongo import MongoClient
from datetime import datetime,timedelta
import re
import warnings
import plotly.express as px
import calendar
from difflib import SequenceMatcher


load_dotenv()

warnings.simplefilter(action='ignore', category=FutureWarning)
MONGO_URI = os.getenv("MONGO_URI")

In [45]:
# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client['Flights']
arrival_collection = db['arrivals_barcelona']
departure_collection = db['departures_barcelona']

In [46]:
#query = {'date' : {"$gt" : "2024-02-26", "$lte" : "2024-02-29" }}

In [47]:
def get_flight_data(start_str=None, end_str=None, type='arrival'):
    
    if start_str and end_str:
        query = {'date' : {"$gt" : start_str, "$lte" : end_str }}
    else:
        query = {}
        
    if(type == 'arrival'):
        collection = db['arrivals_barcelona']
    elif(type == 'departure'):
        collection = db['departures_barcelona']
        
    df_query = pd.DataFrame(list(collection.find(query).sort("date", 1)))
    df = pd.DataFrame()
    
    for document in df_query['flights']:
        df = df.append(document, ignore_index=True)
    
    df['airline'] = df['airline'].str.replace(r' [0-9]*', '')
    
    return df

arrival_df = get_flight_data('2024-01-30', '2024-03-18')
departure_df = get_flight_data('2024-01-30', '2024-03-18', 'departure')
    

In [48]:
def get_time(time_str):
    try:
        match = re.search(r'([0-2][0-9]:[0-5][0-9])', time_str)
        return match.group(0)
    except:
        return None

In [49]:
arrival_df['date'] = arrival_df['date'].apply(lambda x: datetime.strptime(x,'%d %b %Y').date())
departure_df['date'] = departure_df['date'].apply(lambda x: datetime.strptime(x,'%d %b %Y').date())

In [50]:
# Flight analysis given a date range
#Analysis based on arrival or departure.
#Number of flights per airline
#Number of flights to city
#Histogram of flights per day.
#Flights on time
#Flights delayed
#Flights cancelled

In [51]:
#Cleaning arrival_df

arrival_df.drop(arrival_df[arrival_df['status'] == 'Scheduled'].index, inplace = True)
arrival_df.drop(arrival_df[(arrival_df['iata'] == 'MMM')].index, inplace = True)
arrival_df['status'] = arrival_df['status'].str.replace(r'Cancelled.*', 'Cancelled')

arrival_df.reset_index(drop=True, inplace=True)

In [52]:
arrival_df['status'].value_counts()

Unknown            163
Cancelled          126
Estimated          101
Landed 11:00        36
Landed 09:12        33
                  ... 
Estimated 21:56      1
Estimated 09:56      1
Landed 02:08         1
Estimated 00:30      1
Estimated 23:53      1
Name: status, Length: 1496, dtype: int64

In [53]:
arrival_df2 = arrival_df.copy()

In [54]:
arrival_df2['status_hour'] = arrival_df2['status'].apply(lambda x: get_time(x))
arrival_df2

,time,date,iata,arrival/departure,flight,airline,status,status_hour
0,00:05,2024-01-31,BGY,Milan,FR6304,Ryanair,Landed 23:54,23:54
1,00:05,2024-01-31,FLR,Florence,VY6004,Vueling,Landed 23:41,23:41
2,00:05,2024-01-31,LIS,Lisbon,VY8463,Vueling,Landed 23:42,23:42
3,00:10,2024-01-31,VCE,Venice,VY6405,Vueling,Landed 23:47,23:47
4,00:10,2024-01-31,LGW,London,VY7821,Vueling,Landed 23:58,23:58
...,...,...,...,...,...,...,...,...
19340,23:45,2024-03-18,PMI,Palma de Mallorca,WT223,Swiftair,Unknown,None
19341,23:50,2024-03-18,BIO,Bilbao,VY1435,Vueling,Landed 23:42,23:42
19342,23:55,2024-03-18,PMI,Palma de Mallorca,FR3071,Ryanair,Landed 23:33,23:33
19343,23:55,2024-03-18,MAD,Madrid,VY1009,Vueling,Landed 23:48,23:48


In [55]:
time_df = arrival_df2.copy()
arrival_df2['status_hour'] = pd.to_datetime(arrival_df2['status_hour'])

In [56]:
arrival_df2.groupby(arrival_df2['status_hour'].dt.hour).size()
px.bar(arrival_df2.groupby(arrival_df2['status_hour'].dt.hour).size(), title='Number of flights per hour')

In [57]:
time_df = time_df[['time','status_hour']].dropna().apply(pd.to_datetime)
time_df['hour_dif'] = time_df.apply(lambda x: x['status_hour'] - x['time'], axis=1)

In [58]:
def pos_check(x, zero_value=pd.Timedelta(0,'s')):
    return pd.Timedelta(x) >= zero_value

#time_df['IsLate'] = time_df['hour_dif'].apply(lambda x: pos_check(x, zero_value=pd.Timedelta(0,'s')))
#late_df = time_df[(time_df['IsLate'] == True) & (time_df['real_hour'].dt.hour != 23)]['hour_dif'].dt.total_seconds()/60

#late_10 = late_df.apply(lambda x: x // 10)

#late_10.value_counts().sort_index()

late_dict = {
    'on-time' : 10
}

late_dict


{'on-time': 10}

In [59]:
#Clean departure data
departure_df.drop(departure_df[(departure_df['status'] == 'Scheduled') | (departure_df['iata'] == 'MMM')].index, inplace = True)
departure_df['status'] = departure_df['status'].str.replace(r'Cancelled.*', 'Cancelled')
departure_df.reset_index(drop=True, inplace=True)

In [60]:
#Logic for late departures
departure_df['status_time'] = departure_df['status'].apply(lambda x: get_time(x))
time_df = departure_df[['time','status_time']].dropna().apply(pd.to_datetime)
time_df['time_dif'] = time_df.apply(lambda x: x['status_time'] - x['time'], axis=1)

In [61]:
time_df

,time,status_time,time_dif
1,2024-04-01 04:30:00,2024-04-01 04:51:00,0 days 00:21:00
2,2024-04-01 05:35:00,2024-04-01 05:45:00,0 days 00:10:00
3,2024-04-01 05:45:00,2024-04-01 05:53:00,0 days 00:08:00
4,2024-04-01 05:45:00,2024-04-01 05:55:00,0 days 00:10:00
5,2024-04-01 06:00:00,2024-04-01 06:50:00,0 days 00:50:00
...,...,...,...
22650,2024-04-01 22:35:00,2024-04-01 22:32:00,-1 days +23:57:00
22652,2024-04-01 22:40:00,2024-04-01 23:06:00,0 days 00:26:00
22653,2024-04-01 22:50:00,2024-04-01 23:37:00,0 days 00:47:00
22654,2024-04-01 22:55:00,2024-04-01 23:10:00,0 days 00:15:00


In [62]:
time_df['isLate']  = time_df['time_dif'].apply(lambda x: pos_check(x, zero_value=pd.Timedelta(0,'s')))

In [135]:
min_late = time_df[time_df['isLate'] == True]['time_dif'].dt.total_seconds()/60
dec_late = min_late//10
dec_late = dec_late.value_counts().sort_index()

def get_series_range(series,start,end):
    sum = 0
    for i in range(start,end):
        sum += series.get(i,0)
    return sum
    
print(dec_late)
print('On average flights depart late by: ', round(min_late.mean(),2), 'minutes')
print(get_series_range(dec_late,12,int(dec_late.index.max())))

0.0     3300
1.0     7021
2.0     3474
3.0     1565
4.0      788
5.0      449
6.0      257
7.0      161
8.0      103
9.0       73
10.0      45
11.0      51
12.0      33
13.0      12
14.0      12
15.0      12
16.0       8
17.0       3
18.0       6
19.0       5
20.0       2
21.0       2
22.0       1
24.0       2
25.0       1
26.0       1
31.0       1
34.0       3
35.0       2
37.0       1
39.0       1
40.0       1
Name: time_dif, dtype: int64
On average flights depart late by:  22.31 minutes
108


In [120]:
dec_late_dict = {
    '0-10 minutes' : dec_late[0],
    '10-20 minutes' : dec_late[1],
    '20-30 minutes' : dec_late[2],
    '30-60 minutes': dec_late[3:5].sum(),
    '60-120 minutes' : dec_late[6:12].sum(),
    '120+ minutes' : dec_late[13:].sum()
}
#dec_late_dict to dataframe
dec_late_df = pd.DataFrame(dec_late_dict.items(), columns=['Time', 'Count'])

fig = px.pie(dec_late_df,values='Count', names='Time',hole=0.3, title='Departure Delays',color_discrete_sequence=px.colors.sequential.Sunset,)
fig.show()

In [65]:
departure_df['airline'].value_counts()

Vueling                 7663
Ryanair                 3064
easyJet                 1773
Iberia                  1694
WizzAir                  745
                        ... 
HiSkyEurope                1
ArkiaIsraeliAirlines       1
ASLAirlinesBelgium         1
HelveticAirways            1
HumoAir                    1
Name: airline, Length: 125, dtype: int64

In [66]:
airline_df = pd.DataFrame()
airline_df['Arrivals'] = arrival_df['airline'].value_counts()
airline_df['Departures'] = departure_df['airline'].value_counts()
airline_df['Total'] = airline_df['Arrivals'] + airline_df['Departures']
airline_df.reset_index(inplace=True)
airline_df.rename(columns={'index':'Airline'}, inplace=True)
airline_df.fillna(0, inplace=True)
fig = px.bar(airline_df.iloc[:10], title='Number of Flights Per Airline', labels={'value':'Count', 'index':'Airline'}, barmode='stack',x='Airline',y=['Arrivals','Departures'])
fig.show()

In [67]:
location_df = pd.DataFrame()
location_df['Arrivals'] = arrival_df['arrival/departure'].value_counts()
location_df['Departures'] = departure_df['arrival/departure'].value_counts()
location_df['Total'] = location_df['Arrivals'] + location_df['Departures']
location_df.reset_index(inplace=True)
location_df.rename(columns={'index':'Location'}, inplace=True)
location_df.fillna(0, inplace=True)
fig = px.bar(location_df.iloc[:10], title='Most Popular Locations', labels={'value':'Count', 'index':'Location'}, barmode='stack',x='Location',y=['Arrivals','Departures'])
fig.show()

In [68]:
departure_df['status_time'] = departure_df['status'].apply(lambda x: get_time(x))
departure_df['status_time'] = pd.to_datetime(departure_df['status_time'])

df_concurrency = pd.DataFrame()
df_concurrency['Arrivals'] = arrival_df2.groupby(arrival_df2['status_hour'].dt.hour).size()
df_concurrency['Departures'] = departure_df.groupby(departure_df['status_time'].dt.hour).size()

px.bar(df_concurrency,x=df_concurrency.index,y=['Arrivals','Departures'], title='Number of flights per hour')

In [366]:
new_df = departure_df[(departure_df['airline'] == 'Ryanair') & (departure_df['arrival/departure'] == 'Milan')].copy()
compare_df = departure_df[departure_df['arrival/departure'] == 'Milan'].copy()
new_df.dropna(subset=['status_time'], inplace=True)
new_df['time'] = pd.to_datetime(new_df['time'])
new_df['status_time'] = pd.to_datetime(new_df['status'].apply(lambda x: get_time(x)))
new_df['dif'] = new_df['status_time'] - new_df['time']
new_df['isLate'] = new_df['dif'].apply(lambda x: pos_check(x))
late_time = new_df[new_df['isLate'] == True]['dif'].dt.total_seconds()/60//10
new_df['weekday'] = new_df['date'].apply(lambda x: x.weekday())
new_df['time'] = new_df['time'].apply(lambda x: x.strftime('%H:%M'))

compare_df['weekday'] = compare_df['date'].apply(lambda x: calendar.day_name[x.weekday()])

#create a dataframe with a row with the airline, time, weekday, iata, arrival/departure, airline from dummy data
selected_df = pd.DataFrame(columns=['time','weekday','arrival/departure','airline'])
selected_df.loc[0] = ['12:00','Monday','Milan','']

compare_df = compare_df[['time','weekday','arrival/departure','airline']]

# or diff_df = df2 - df1.iloc[0, :]

In [396]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

def normalize_df(df):
    df = df[['time','weekday','iata','arrival/departure','airline']].copy()
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.hour * 60 + df['time'].dt.minute
    df = encode(df,'weekday',7)
    df.drop('weekday', axis=1, inplace=True)
    return df

def similar(a, b):
    try:
        return SequenceMatcher(None, a, b).ratio()
    except:
        return 0

row_df = pd.DataFrame(columns=['time','weekday','iata','arrival/departure','airline'])
row_df.loc[0] = ['10:00',6,'MXP','Milan','Ryanair']

compare_df = new_df.drop_duplicates().sample(20)

pprint.pprint(row_df)
pprint.pprint(compare_df[['time','weekday','iata','arrival/departure','airline']])

row_df = normalize_df(row_df)
compare_df = normalize_df(compare_df)

def similarity(row_df, compare_df):
    ponderation = {
        'time' : .3,
        'weekday' : .5,
        'iata' : .1,
        'airline' : .1,
    }
    similar_df = pd.DataFrame(columns=['time','weekday','iata','arrival/departure','airline'])
    similar_df['time'] = compare_df['time'].apply(lambda x: abs(x - row_df['time'].iloc[0]))/1440*ponderation['time']
    similar_df['weekday_cos'] = compare_df['weekday_cos'].apply(lambda x: abs(x - row_df['weekday_cos'].iloc[0]))
    similar_df['weekday_sin'] = compare_df['weekday_sin'].apply(lambda x: abs(x - row_df['weekday_sin'].iloc[0]))
    similar_df['weekday'] = (similar_df['weekday_cos'] + similar_df['weekday_sin'])/2.740174*ponderation['weekday']
    similar_df['iata'] = compare_df['iata'].apply(lambda x: 1 - similar(x, row_df['iata'].iloc[0])*ponderation['iata'])//1
    similar_df['airline'] = compare_df['airline'].apply(lambda x: 1 - similar(x, row_df['airline'].iloc[0]))*ponderation['airline']//1
    similar_df['iata'] = similar_df['iata']*ponderation['iata']
    similar_df['airline'] = similar_df['airline']*ponderation['airline']
    similar_df.drop(['weekday_cos','weekday_sin'], axis=1, inplace=True)
    #Add sum of rows
    similar_df['score'] = similar_df.sum(axis=1)
    similar_df.sort_values(by='score', ascending=True, inplace=True)
    return similar_df.index

#Most important features weekday, arrival/departure, time, iata
#What will be compared in order of importance.
#Weekday, time, iata, airline.

    time  weekday iata arrival/departure  airline
0  10:00        6  MXP             Milan  Ryanair
        time  weekday iata arrival/departure  airline
9675   22:10        1  BGY             Milan  Ryanair
10129  01:05        3  BGY             Milan  Ryanair
3121   22:10        1  BGY             Milan  Ryanair
7794   23:05        4  MXP             Milan  Ryanair
11399  13:45        5  BGY             Milan  Ryanair
16344  11:25        1  BGY             Milan  Ryanair
2243   21:55        6  BGY             Milan  Ryanair
7283   06:45        4  BGY             Milan  Ryanair
5431   21:55        6  BGY             Milan  Ryanair
19949  13:50        2  BGY             Milan  Ryanair
11163  23:05        4  MXP             Milan  Ryanair
700    16:05        3  BGY             Milan  Ryanair
20123  20:20        2  MXP             Milan  Ryanair
4059   06:45        4  BGY             Milan  Ryanair
10648  06:45        4  BGY             Milan  Ryanair
3750   12:00        3  BGY          

In [399]:
# 0  10:00        6  BVA             Milan  Ryanair
recommendations = new_df.loc[similarity(row_df, compare_df).index]
recommendations[['time','weekday','iata','arrival/departure','airline']]
#compare_df.loc[indexes]

,time,weekday,iata,arrival/departure,airline
9675,22:10,1,BGY,Milan,Ryanair
10129,01:05,3,BGY,Milan,Ryanair
3121,22:10,1,BGY,Milan,Ryanair
7794,23:05,4,MXP,Milan,Ryanair
11399,13:45,5,BGY,Milan,Ryanair
16344,11:25,1,BGY,Milan,Ryanair
2243,21:55,6,BGY,Milan,Ryanair
7283,06:45,4,BGY,Milan,Ryanair
5431,21:55,6,BGY,Milan,Ryanair
19949,13:50,2,BGY,Milan,Ryanair
